In [ ]:
# %% Deep learning - Section 7.56
#    Defining models using sequential vs. class

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F

from google.colab                     import files
from torchsummary                     import summary
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

nClust = 100
blur   = 1

A = [1,1]
B = [5,1]

a = [ A[0]+np.random.randn(nClust)*blur, A[1]+np.random.randn(nClust)*blur ]
b = [ B[0]+np.random.randn(nClust)*blur, B[1]+np.random.randn(nClust)*blur ]

# True labels
labels_np = np.vstack(( np.zeros((nClust,1)), np.ones((nClust,1)) ))

# Concatenate
data_np = np.hstack((a,b)).T

# Convert into torch tensor
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()


In [ ]:
# Plotting

fig = plt.figure(figsize=(7,7))

plt.plot( data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs' )
plt.plot( data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ro' )
plt.title('Some binary data')
plt.xlabel('x1')
plt.ylabel('x2')

plt.savefig('figure78_sequential_vs_class.png')

plt.show()

files.download('figure78_sequential_vs_class.png')


In [ ]:
# Build model using nn.Sequential

ANNclassify = nn.Sequential(
                 nn.Linear(2,1),
                 nn.ReLU(),
                 nn.Linear(1,1),
                 #nn.Sigmoid()
                 )


In [ ]:
# Build model using a class

class Class4ANN(nn.Module):
    def __init__(self):
        super().__init__()

        # Input layer
        self.input = nn.Linear(2,1)

        # Output layer
        self.output = nn.Linear(1,1)

    # Forward propagation
    def forward(self,x):

        # Pass through input layer
        x = self.input(x)

        # Apply ReLU
        x = F.relu(x)

        # Output layer
        x = self.output(x)
        x = torch.sigmoid(x)

        return x

# Create an instance of the class
ANNclassify = Class4ANN()


In [ ]:
# Other model features (learning rate, loss function and optimizer)

learn_rate = 0.01
loss_fun   = nn.BCELoss()
optimizer  = torch.optim.SGD(ANNclassify.parameters(),lr=learn_rate)


In [ ]:
# Train model

num_epochs = 1000
losses     = torch.zeros(num_epochs)

for epoch_i in range(num_epochs):

    # Forward propagation
    yHat = ANNclassify(data)

    # Compute loss
    loss = loss_fun(yHat,labels)
    losses[epoch_i] = loss

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# Plotting

plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Losses over epoch')

plt.savefig('figure79_sequential_vs_class.png')

plt.show()

files.download('figure79_sequential_vs_class.png')


In [ ]:
# Compute predictions

# Final forward pass
predictions = ANNclassify(data)
pred_labels = predictions > 0.5

# Find errors
misclassified = np.where(pred_labels != labels)[0]

# Accuracy
tot_acc = 100-100*len(misclassified)/(2*nClust)
print(f'Total accuracy: {tot_acc}%')


In [ ]:
# Plotting

# Show labeled data
fig = plt.figure(figsize=(7,7))

plt.plot( data[misclassified,0],data[misclassified,1],'rx',markersize=12,markeredgewidth=3 )
plt.plot( data[np.where(~pred_labels)[0],0],data[np.where(~pred_labels)[0],1],'s' )
plt.plot( data[np.where(pred_labels)[0],0], data[np.where(pred_labels)[0],1],'o'  )

plt.legend(['Misclassified','Group 1','Group 2'])
plt.xlabel('x1')
plt.ylabel('x2')
plt.title(f'Overall accuracy: {tot_acc}%')

plt.savefig('figure80_sequential_vs_class.png')

plt.show()

files.download('figure80_sequential_vs_class.png')
